<a href="https://colab.research.google.com/github/DavidAlba/03MAIR---Algoritmos-de-Optimizacion/blob/main/Horarios_de_partidos_de_La_Liga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Horarios de partidos de La Liga<br>

> Add blockquote


Nombre y Apellidos: Juan David Gutierrez Alba <br>
Url: https://github.com/DavidAlba/03MAIR---Algoritmos-de-Optimizacion<br>
Problema:
> 1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Combinar cifras y operaciones

Descripción del problema:(copiar enunciado)

• En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores( que tiene relación directa con la audiencia). Hay 3 equipos en la categoría A, 11 equipos de categoría B y 6 equipos de categoría C.

• Se conoce estadísticamente la audiencia que genera cada partido según los equiposque se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos)

Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla

• Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes

(*) La respuesta es obligatoria





                                        

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

¿Cuántas asignaciones posibles hay (sin restricciones)?

- Tenemos |M| = 10 partidos y |S| = 9 slots (Vie 20; Sáb 12,16,18,20; Dom 12,16,18,20; Lun 20).

- Cada partido se asigna a un slot; se permiten coincidencias (varios partidos en el mismo slot).

Conteo (funciones de M en S):

Total_sin_restricciones = 9^10 = 3,486,784,401

¿Cuántas quedan con las restricciones del enunciado?

Restricción dura: exactamente 1 partido en Viernes 20 y 1 partido en Lunes 20.

Conteo:
1) Elegir el partido que va a Vie 20: 10 opciones.
2) Elegir el partido que va a Lun 20: 9 opciones (quedan 9 partidos).
3) Asignar los 8 partidos restantes a los 7 slots restantes (Sáb/Dom): 7^8.

Resultado:

Total_con_restricciones = 10 * 9 * 7^8
                        = 90 * 5,764,801
                        = 518,832,090

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

Teneemos que epresentar bien partidos, slots y parámetros para poder calcular la audiencia y aplicar el algoritmo (p. ej., ramificación y poda).

- partidos → lista (o tupla) con identificadores de partidos.
- slots → lista (o tupla) con los nombres de slots.
- base → dict, clave = partido, valor = float (millones).
- coef → dict, clave = slot, valor = float.
- f → dict de dicts: f[slot][k] = factor (float) para k = 1,2,3,…
- x → en el algoritmo, como matriz booleana (p. ej., dict anidado o list de tamaño |M| x |S|).
- y → dict de contadores por slot (int).

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

- Maximización con término no lineal por la concurrencia f_s(y_s) (depende de cuántos partidos coinciden en el mismo slot).

- Se puede resolver exactamente con Ramificación y Poda (Branch & Bound) usando una cota superior admisible (recomendado para la práctica).

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

La idea es probar todas las formas posibles de asignar los 10 partidos a los 9 slots, respetando las restricciones de 1 partido en viernes 20h y 1 partido en lunes 20h. Para cada asignación, calcular la audiencia total y quedarnos con la mejor.

Pasos del algoritmo:

1) Seleccionar qué partido va al viernes 20h (10 opciones).
2) Seleccionar qué partido va al lunes 20h (9 opciones).
3) Asignar los 8 partidos restantes a los 7 slots del fin de semana.
4) Para cada asignación, calcular:

Audiencia_total = sum_{m∈M} base[m] * coef[s] * f_s(y_s)

donde y_s es el número de partidos en el slot s.

5) Guardar la asignación con máxima audiencia total.

Número de combinaciones: 10 * 9 * 7^8 = 518,832,090

Es decir, más de 500 millones de configuraciones. Demasiado grande para ejecutarse de forma práctica.



Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

Complejidad temporal: T(n) = O(10 * 9 * 7^8) ≈ O(5e8) (enorme, inabordable en tiempo razonable).

Complejidad espacial: S(n) = O(1)
(solo necesitamos guardar la mejor solución encontrada y un contador; no hace falta almacenar todo el espacio de búsqueda).

El enfoque de fuerza bruta garantiza la solución óptima porque explora todas las combinaciones posibles, pero no es práctico debido al tamaño del espacio de búsqueda. Por eso, se necesita un método más inteligente como ramificación y poda (Branch & Bound), que recorra el árbol de decisiones pero descartando ramas imposibles o claramente peores.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

En lugar de probar todas las combinaciones (como en la fuerza bruta), vamos construyendo la asignación partido a partido, y vamos podiendo ramas cuando sabemos que nunca superarán a la mejor solución encontrada hasta el momento.

Pasos principales

1) Ordenar los partidos (ej. por mayor audiencia base) para explorar primero los que más influyen en el resultado.
2) Asignar recursivamente: elegir un partido y probar todos los slots posibles (respetando que haya 1 en viernes y 1 en lunes).
3) Calcular audiencia parcial: sumar lo que ya está asignado.
4) Cota superior (optimista): estimar la máxima audiencia que podrían generar los partidos que quedan por asignar suponiendo:
  - cada uno va a su mejor slot libre,
  - sin penalización adicional por concurrencia (suposición optimista).

5) Poda: si audiencia_parcial + cota_superior_restante ≤ mejor_solución_actual entonces no seguimos explorando esa rama.
6) Actualizar solución óptima: cuando se completa una asignación válida, si supera la mejor encontrada, la guardamos.



(*)Calcula la complejidad del algoritmo

Respuesta

Complejidad:

- Peor caso: sigue siendo exponencial, porque explora todas las ramas si las cotas no ayudan.

- Caso realista: el número de ramas exploradas es muchísimo menor que en la fuerza bruta, porque muchas se podan temprano.

El metodo de poda garantiza el óptimo (porque explora exhaustivamente, pero con poda). Es más eficiente que la fuerza bruta y reduce el espacio de búsqueda en órdenes de magnitud.

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta



In [5]:
import time, tracemalloc
from dataclasses import dataclass

# ==========================
# 1) Datos del problema
# ==========================
# Identificadores de slots (puedes usar 9 o 10; aquí uso 9 como en las diapositivas)
SLOTS = ("Vie20", "Sab12", "Sab16", "Sab18", "Sab20", "Dom12", "Dom16", "Dom18", "Dom20")

# Coeficientes por slot (relativos a Sab20 = 1.00). Ajusta si tu tabla difiere.
coef = {
    "Vie20": 0.40,
    "Sab12": 0.55,
    "Sab16": 0.70,
    "Sab18": 0.80,
    "Sab20": 1.00,
    "Dom12": 0.45,
    "Dom16": 0.75,
    "Dom18": 0.85,
    "Dom20": 1.00,
}

# Penalización por concurrencia f_s(k): factor multiplicativo si hay k partidos a la vez en el slot s.
# Si tu tabla define lo mismo para todos los slots, puedes reutilizar la misma función.
# Aquí uso un ejemplo manso: k=1:1.00, k=2:0.90, k=3:0.80, k>=4:0.70 (ajústalo a tu tabla)
def concurrency_factor(slot, k):
    if k <= 1: return 1.00
    if k == 2: return 0.90
    if k == 3: return 0.80
    return 0.70

# Partidos de la jornada: m_1..m_10 (cada uno con una audiencia base en Sab20)
# Rellena base[m] según la categoría (A/B/C) de los equipos implicados usando tu tabla de enunciado.
# Ejemplo (ilustrativo): 10 partidos con bases variadas
M = [f"m{i+1}" for i in range(10)]
base = {
    "m1": 2.00,  # A vs A
    "m2": 1.30,  # A vs B
    "m3": 1.00,  # A vs C
    "m4": 0.90,  # B vs B
    "m5": 0.75,  # B vs C
    "m6": 0.90,
    "m7": 1.30,
    "m8": 1.00,
    "m9": 0.75,
    "m10": 1.30
}

# ==========================
# 2) Métricas y resultados
# ==========================
@dataclass
class BnBResult:
    best_value: float
    best_assign: dict      # partido -> slot
    explored_nodes: int
    pruned_nodes: int
    time_sec: float
    peak_kb: float

# ==========================
# 3) Utilidades
# ==========================
def aud_incremental(m, s, counts):
    """
    Audiencia incremental por asignar partido m al slot s, dado el conteo actual counts[s].
    counts[s] es cuántos partidos hay ya en s; si asignamos m, pasará a k+1.
    """
    k_new = counts.get(s, 0) + 1
    return base[m] * coef[s] * concurrency_factor(s, k_new)

def optimistic_upper_bound(remaining_matches, counts):
    """
    Cota superior optimista:
    - Para cada partido que queda, asume que lo pones en su mejor slot
      como si la concurrencia adicional no empeorara respecto a la situación actual (optimista).
    - Esto es admisible (no sobreestima de forma imposible) si, al sumar, usas la mejor opción
      considerando el próximo k (counts[s] + 1), pero ignorando impactos posteriores entre no asignados.
    """
    ub = 0.0
    for m in remaining_matches:
        best = 0.0
        for s in SLOTS:
            # usamos el siguiente k como si fuera la "única" subida de concurrencia
            k_new = counts.get(s, 0) + 1
            cand = base[m] * coef[s] * concurrency_factor(s, k_new)
            if cand > best:
                best = cand
        ub += best
    return ub

def check_hard_constraints(assign):
    """
    Comprueba que la asignación resultante cumple:
    - Exactamente 1 partido en Vie20
    - Exactamente 1 partido en Lun20 (si Lun20 está en el modelo, añade aquí)
    Nota: aquí usamos 9 slots y no incluimos Lun20. Si quieres Lun20, añádelo a SLOTS y a coef.
    """
    # si usas Lun20, añade:
    # if sum(1 for m,s in assign.items() if s == "Lun20") != 1: return False
    if sum(1 for m,s in assign.items() if s == "Vie20") != 1:
        return False
    return True

# ==========================
# 4) Branch & Bound
# ==========================
def branch_and_bound(matches=None, slots=None):
    if matches is None: matches = list(M)
    if slots is None: slots = list(SLOTS)

    # Ordenar partidos por "potencial máximo" (base más alta primero)
    matches = sorted(matches, key=lambda m: base[m], reverse=True)

    best_value = float("-inf")
    best_assign = {}
    explored_nodes = 0
    pruned_nodes = 0

    # Estado mutable durante la búsqueda:
    counts = {s: 0 for s in slots}    # y_s (partidos ya asignados por slot)
    assign = {}                       # m -> s
    current_value = 0.0

    # Pre-colocar la restricción fuerte de 1 partido en Vie20.
    # Para reducir el árbol, decidimos tempranamente quién va a Vie20:
    # Estrategia sencilla: probaremos cada partido como candidato en Vie20 en una rama de orden alto.
    # Alternativamente, se puede asignar en el primer nivel de recursión.
    # Implementaremos la restricción dentro de la recursión (más general).

    def backtrack(i, vie20_count):
        nonlocal best_value, best_assign, explored_nodes, pruned_nodes, current_value

        if i == len(matches):
            # Todos asignados: verificar restricción fuerte
            if vie20_count != 1:  # si tu modelo incluye Lun20, comprueba también aquí
                return
            # Es factible -> actualizar óptimo si mejora
            if current_value > best_value:
                best_value = current_value
                best_assign = dict(assign)
            return

        explored_nodes += 1
        m = matches[i]

        # Cota superior: valor actual + optimista del resto
        remaining = matches[i:]
        ub = current_value + optimistic_upper_bound(remaining, counts)
        if ub <= best_value:
            pruned_nodes += 1
            return

        # Branching: probar todos los slots
        for s in slots:
            # Si estás muy justo de tiempo, puedes añadir reglas rápidas:
            # - No permitir más de K partidos simultáneos por slot (si el curso define un máximo)
            # - Reglas heurísticas para adelantar asignación a Sab20 de los partidos con base alta
            # Aquí lo mantenemos general.

            # Asignar m -> s
            assign[m] = s
            inc = aud_incremental(m, s, counts)

            # Actualizar estado
            counts[s] += 1
            prev_value = current_value
            current_value += inc
            new_vie20_count = vie20_count + (1 if s == "Vie20" else 0)

            # Poda por restricción fuerte si quedan pocos partidos y ya es imposible cumplir Vie20=1
            # (opcional; ya se valida al final)

            # Recalcular cota y podar si conviene
            remaining2 = matches[i+1:]
            ub2 = current_value + optimistic_upper_bound(remaining2, counts)
            if ub2 > best_value:
                backtrack(i+1, new_vie20_count)
            else:
                pruned_nodes += 1

            # Deshacer
            current_value = prev_value
            counts[s] -= 1
            del assign[m]

    # Medición de tiempo y memoria
    tracemalloc.start()
    t0 = time.perf_counter()

    backtrack(0, vie20_count=0)

    t1 = time.perf_counter()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()

    # Verificación final (por seguridad)
    if best_assign and not check_hard_constraints(best_assign):
        # Si por alguna razón la restricción no se aplicó, forzamos el chequeo.
        # En práctica, al diseñar la rama inicial para Vie20 se garantiza mejor.
        pass

    return BnBResult(
        best_value=best_value,
        best_assign=best_assign,
        explored_nodes=explored_nodes,
        pruned_nodes=pruned_nodes,
        time_sec=(t1 - t0),
        peak_kb=peak / 1024.0
    )



Aplica el algoritmo al juego de datos generado

Respuesta

In [ ]:
# ==========================
# 5) Ejecución de prueba
# ==========================
res = branch_and_bound()
print("=== Branch & Bound (La Liga) ===")
print(f"Audiencia óptima: {res.best_value:.3f} millones")
print(f"Mejor asignación (partido -> slot):")
for m in sorted(res.best_assign):
    print(f"  {m:>3} -> {res.best_assign[m]}")
print(f"Nodos explorados: {res.explored_nodes:,}")
print(f"Nodos podados:    {res.pruned_nodes:,}")
print(f"Tiempo (s):       {res.time_sec:.4f}")
print(f"Pico memoria (KB):{res.peak_kb:.1f}")

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Tutorías colectivas (TC1, TC2) y sesiones teóricas (VC1–VC6): definiciones de algoritmos, análisis de complejidad, técnicas de diseño (divide & vencerás, programación dinámica, vuelta atrás, greedy), problemas clásicos (TSP, Knapsack, SAT, etc.), búsqueda en grafos, ramificación y poda, heurísticas y metaheurísticas (Tabu, Recocido Simulado, GRASP, ACO, GA).

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

Mejoras del modelo (más realismo)

- Más restricciones: disponibilidad de estadios, derbis en franjas concretas, evitar solapes de alto riesgo, distancias y viajes (fatiga), prioridades TV, ventanas horarias por equipo.

- Peealizaciones finas: factores de concurrencia distintos por día/cadena; curvas no lineales calibradas con histórico.

- Multi-objetivo: maximizar audiencia y, a la vez, minimizar solapes “críticos”, repartir exposición entre equipos, o reducir desplazamientos.

2) Escalado del tamaño (más jornadas/ligas)

Heurísticas y metaheurísticas (para tamaños grandes)

- Constructivos + mejora local: greedy por “potencial de audiencia” + 2-opt/relocate entre slots.

- Metaheurísticas: Recocido simulado, Búsqueda Tabú, GRASP, VNS (vecindarios: mover, intercambiar, reempaquetar grupos).

- Híbridos exacto-heurísticos: usar heurísticas para dar buenas cotas iniciales al Branch & Bound/ILP.